# Data Augmentation

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from tqdm import tqdm
import pandas as pd
from wordcloud_fa import WordCloudFa

from hazm import word_tokenize
from nltk.util import ngrams

In [2]:
tqdm.pandas()

In [3]:
base_path = "../data/"

tweets_path = base_path + 'remote_tweets.csv'

In [4]:
tweets_df = pd.read_csv(tweets_path, low_memory=False)
tweets_df.head()

,conversation_id,retweets_count,mentions,created_at,likes_count,in_reply_to_user_id,_id,source,user_id,in_reply_to_status_id,tweet,replies_count,quoted_status_id,hashtags,cashtags,urls
0,1375557622410149894,2,NaN,2021-03-26 21:23:21+00:00,111,572051429.0,1375559020451356674,Twitter for iPhone,572051429,1.375558e+18,به من گزارش میده حقوقش میدونم سالی ۸۵تا، همسرش...,3,NaN,NaN,NaN,NaN
1,1375559420176908296,3,NaN,2021-03-26 21:24:57+00:00,163,NaN,1375559420176908296,Twitter for iPhone,812764546012246017,NaN,اینایی که رفتن سفر، مهمونی و... هیچ پروتکلی هم...,1,NaN,NaN,NaN,NaN
2,1383828354197979136,2,NaN,2021-04-18 17:02:44+00:00,37,NaN,1383828354197979136,Twitter for Android,985101690536898565,NaN,دارم فکر میکنم که چقدر خسته ام با اینکه امروز ...,3,NaN,NaN,NaN,NaN
3,1383840315031781385,0,NaN,2021-04-18 17:50:16+00:00,11,NaN,1383840315031781385,Twitter Web App,310871673,NaN,یک سال و نیمه از قبل آبان ۹۸ دورکار شدم تا الان,1,1.383657e+18,NaN,NaN,NaN
4,1383820625354002439,1,['15782990'],2021-04-18 17:31:42+00:00,9,15782990.0,1383835643239747593,Twitter Web App,1271095835028307969,1.383821e+18,@shahrzaad حتمن میرفتم به شهری که به اقوام یا ...,1,NaN,NaN,NaN,NaN


In [5]:
def get_uni_bi_tri_grams(text):
    tokens = word_tokenize(text)
    bigrams = list(ngrams(tokens, 2)) if len(tokens) >= 2 else []
    bigram_tokens = [' '.join(b) for b in bigrams]
    trigrams = list(ngrams(tokens, 3)) if len(tokens) >= 3 else []
    trigram_tokens = [' '.join(t) for t in trigrams]
    tokens = tokens + bigram_tokens + trigram_tokens
    return tokens

In [6]:
tweets_df['tokens'] = tweets_df['tweet'].progress_apply(lambda t: get_uni_bi_tri_grams(t))

100%|███████████████████████████████████████████████████████████████████████████| 45359/45359 [00:06<00:00, 7102.25it/s]


In [10]:
def get_word_counts(data_df):
    word_counts = {}
    for index, row in tqdm(data_df.iterrows()):
        for token in row['tokens']:
            if token in word_counts:
                word_counts[token] += 1
            else:
                word_counts[token] = 1
    return word_counts

tweet_word_counts = get_word_counts(tweets_df)

45359it [00:07, 6064.79it/s]


In [12]:
sorted_words = dict(sorted(tweet_word_counts.items(), key=lambda item: item[1], reverse=True))
sorted_words

{'.': 57676,
 'و': 42696,
 'دورکاری': 29595,
 '،': 24923,
 'از': 24256,
 'که': 23012,
 'به': 22267,
 ':': 21786,
 'کار': 20153,
 'در': 19825,
 'رو': 11784,
 'هم': 11682,
 'با': 11457,
 '//t': 11331,
 ': //t': 11331,
 '//t .': 11331,
 ': //t .': 11331,
 'این': 11026,
 ')': 10596,
 'https': 10533,
 'https :': 10533,
 'https : //t': 10533,
 '. .': 9092,
 'برای': 8971,
 'من': 7860,
 'تو': 7526,
 '؟': 7223,
 'یه': 6793,
 '!': 6187,
 'دورکار': 5574,
 'تا': 5560,
 ') )': 4905,
 'کار در': 4857,
 'خونه': 4780,
 '. . .': 4471,
 'یا': 4411,
 'یک': 4156,
 'منزل': 4155,
 'ما': 4127,
 'را': 3973,
 'خیلی': 3941,
 'ولی': 3930,
 'شرکت': 3888,
 'باید': 3837,
 'در منزل': 3763,
 'کرونا': 3638,
 '#دورکاری': 3413,
 '(': 3331,
 'دیگه': 3281,
 'روز': 3240,
 'است': 3239,
 'کار در منزل': 3207,
 'چاپ': 3187,
 'بعد': 3036,
 'همه': 3003,
 ': )': 2924,
 'ها': 2906,
 'خانه': 2889,
 'نیست': 2877,
 'کنم': 2832,
 'بود': 2826,
 'انجام': 2743,
 'شده': 2742,
 'می': 2737,
 'میشه': 2726,
 'داره': 2675,
 'های': 2673,
 ') ) )

In [13]:
with pd.option_context('display.max_colwidth', 800):
        print(tweets_df['tweet'].head(20))


0                             به من گزارش میده حقوقش میدونم سالی ۸۵تا، همسرش هم دانشجو....خودش سال پیش دکترا گرفته و اومده شرکت ما...احتمالن مشاور مورگج براش مدارک حقوق دریافتی بالا از جایی درست کرده و درخواست وام ۹۵۰تاییش قبول شده...ریسکش جالب بود،مطمینه که قراردادش تمدید میشه و کار از خونه ادامه پیدا میکنه
1                                                                                                                                                                                                    اینایی که رفتن سفر، مهمونی و... هیچ پروتکلی هم رعایت نکردن، فردا میان سرکار میگن خطرناکه و دورکار بشیم بهتره :))
2     دارم فکر میکنم که چقدر خسته ام با اینکه امروز دورکاری داشتم.تو شهرهای بزرگ سروصدا شب ازصبح آزاردهنده تره.متاسفانه نبودفرهنگ شهرنشینی و آپارتمان نشینی بیداد میکنه. بیاید همگی رعایت حال همدیگر رو بکنیم.چاردیواری اختیاری است!بله!اما مردمداری گوشه ای ازفرهنگ وشعور وانسانیته!شب خوش🌃  https://t.co/rEQ08FtHm3
3                                                                     